In [ ]:
#We will attempt to perform content based filtering using 1. overview and 2. Keywords, genres, cast and crew
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

meta = pd.read_csv("movies_metadata_preprocessed.csv")
print(meta['overview'].head())
meta['overview'] = meta['overview'].fillna('')

count = TfidfVectorizer(stop_words='english')
matrix = count.fit_transform(meta['overview'])
cosine_sim = linear_kernel(matrix, matrix)

indices = pd.Series(meta.index, index=meta['title']).drop_duplicates()

In [ ]:
print(cosine_sim)

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return meta['title'].iloc[movie_indices]


rec = get_recommendations("The Conjuring")
print(rec)

In [ ]:
meta = pd.read_csv("movies_metadata_preprocessed.csv")
credits = pd.read_csv('../credits.csv')
meta = meta.merge(credits, on = 'id')
meta = meta.dropna(subset = ['cast', 'crew'])
print(meta)

In [ ]:
from ast import literal_eval
print(type(meta['cast'][0]))
print(type(meta['crew'][0]))
print(type(meta['keywords'][0]))
print(type(meta['genres'][0]))
meta = meta.dropna(subset = ['keywords'])
meta = meta.dropna(subset = ['genres'])  

In [ ]:
#Content based filtering using keywords, genres, cast and crew
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval


features = ['cast','crew','keywords', 'genres']

for feature in features:
    meta.loc[:,feature] = meta[feature].apply(literal_eval)


In [ ]:
import numpy as np
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x,feature):
    if isinstance(x, list):
        if feature!='keywords':
            names = [i['name'] for i in x]
        else:
            names = [i for i in x]
        if len(names) > 3:
            names = names[:3]
        return names  
    return []



def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

meta['director'] = meta['crew'].apply(get_director)

features = ['cast', 'keywords','genres']

for feature in features:
    meta.loc[:,feature] = meta[feature].apply(get_list,args=(feature,))
    print('getlist',feature)
    
# meta.loc[:,'genres']=meta.loc[:,'genres'].apply(get_list)
    
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    meta[feature] = meta[feature].apply(clean_data)
    print('cleandata',feature)
    
def combine(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
meta.loc[:,'combined'] = meta.apply(combine, axis=1)


count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(meta['combined'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

meta = meta.reset_index()
indices = pd.Series(meta.index, index=meta['title'])

In [ ]:
get_recommendations('The Conjuring', cosine_sim2)